# Классификация текстов

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Наша задача обучить модель классифицировать комментарии на позитивные и негативные.

In [1]:
#импортируем библиотеки
import pandas as pd
import spacy
import re

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE=898

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv') #загрузка данных
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.shape

(159292, 3)

In [4]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

У нас 159292 классифицированных комментариев, пропусков нет. Виден дисбаланс классов. Нетоксичных комментариев в 9 раз больше, чем токсичных.

In [6]:
nlp = spacy.load("en_core_web_sm") #создание класса лемматизации

In [7]:
def clear_text(text): #функция очистки текста от лишних символов
    text = text.lower()
    text = re.sub(r'[^a-z ]', ' ', text) 
    return (" ".join(text.split()))

In [8]:
def lemmatize(text): #фукция лемматизации
    text = clear_text(text)
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

In [9]:
#проводим лемматизацию
tqdm.pandas()
data['lemm']= data['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [10]:
data['lemm'].head(10)

0    explanation why the edit make under my usernam...
1    d aww he match this background colour I m seem...
2    hey man I m really not try to edit war it s ju...
3    more I can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
5    congratulation from I as well use the tool wel...
6         cocksucker before you piss around on my work
7    your vandalism to the matt shirvington article...
8    sorry if the word nonsense be offensive to you...
9    alignment on this subject and which be contrar...
Name: lemm, dtype: object

Мы подготовили данные к обучению. Очистили и лемматизировали тексты.

## Обучение моделей

In [11]:
#разбиение на выборки
X_train, X_valid_test, y_train, y_valid_test = train_test_split(
    data['lemm'],
    data['toxic'],
    test_size = 0.4, 
    random_state = RANDOM_STATE,
    stratify = data['toxic'])

X_valid, X_test, y_valid, y_test = train_test_split(
    X_valid_test,
    y_valid_test,
    test_size = 0.5, 
    random_state = RANDOM_STATE,
    stratify = y_valid_test)

In [12]:
count_tf_idf = TfidfVectorizer() #расчет величины TF-IDF
tf_idf = count_tf_idf.fit_transform(X_train)
tf_idf_valid = count_tf_idf.transform(X_valid)
tf_idf_test = count_tf_idf.transform(X_test)

**LogisticRegression**

In [13]:
c_best=5
best_score=0
for c in range(5, 15):
    lr_model = LogisticRegression(C=c).fit(tf_idf, y_train)
    preds = lr_model.predict(tf_idf_valid)
    if f1_score(y_valid, preds) > best_score:
        c_best = c
        best_score = f1_score(y_valid, preds)
        best_model_lr = lr_model
print('C=', c_best, 'score=', best_score) 

C= 11 score= 0.7812660833762223


Модель логистической регрессии с параметром С=11 дает метрику F1 0.78. Подходит под условия задачи.

**KNeighborsClassifier**

In [14]:
for k in range(6,11):
    knn_model = KNeighborsClassifier(n_neighbors=k).fit(tf_idf, y_train)
    preds = knn_model.predict(tf_idf_valid)
    print('n_neighbors=', k, 'score=', f1_score(y_valid, preds))

n_neighbors= 6 score= 0.2292455893494967
n_neighbors= 7 score= 0.21593026750826572
n_neighbors= 8 score= 0.32047773077880076
n_neighbors= 9 score= 0.5499999999999999
n_neighbors= 10 score= 0.5320794148380356


Метод к ближайших соседий дает очень низкую метрику. Не подходит для решения задачи.

**DecisionTreeClassifier**

In [15]:
for min_samples_split in range(2, 5):
    for min_samples_leaf in range(2, 5):
            model = DecisionTreeClassifier(
                random_state=RANDOM_STATE,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf
            )
            model.fit(tf_idf, y_train)
            preds = model.predict(tf_idf_valid)
            print('min_samples_split=', min_samples_split, 'min_samples_leaf=', min_samples_leaf, 'score=', f1_score(y_valid, preds))

min_samples_split= 2 min_samples_leaf= 2 score= 0.6953664147223135
min_samples_split= 2 min_samples_leaf= 3 score= 0.6870179948586117
min_samples_split= 2 min_samples_leaf= 4 score= 0.6936026936026937
min_samples_split= 3 min_samples_leaf= 2 score= 0.6953664147223135
min_samples_split= 3 min_samples_leaf= 3 score= 0.6870179948586117
min_samples_split= 3 min_samples_leaf= 4 score= 0.6936026936026937
min_samples_split= 4 min_samples_leaf= 2 score= 0.6953664147223135
min_samples_split= 4 min_samples_leaf= 3 score= 0.6870179948586117
min_samples_split= 4 min_samples_leaf= 4 score= 0.6936026936026937


Метрика, которую показывает дерево решений, не удовлетворяет условиям задачи.

**Метод опорных векторов**

In [16]:
model_svc = SVC()
model_svc.fit(tf_idf, y_train)
preds = model_svc.predict(tf_idf_valid)
print('score=', f1_score(y_valid, preds))

score= 0.7485185185185186


Метод опорных векторов с начальными параметрами показывает метрику, близкую к нужной нам. Изменим значение ядра. Попробуем линейное ядро.

In [17]:
model_svcl = SVC(kernel='linear')
model_svcl.fit(tf_idf, y_train)
preds = model_svcl.predict(tf_idf_valid)
print('score=', f1_score(y_valid, preds))

score= 0.7693123563726356


Мы получили метрику F1=0.77. Это удовлетворяет условию задачи.

Лучшая метрика у модели логистической регрессии с параметром С=11. Проверим на тестовой выборке.

In [18]:
preds = best_model_lr.predict(tf_idf_test)
print('score=', f1_score(y_test, preds))

score= 0.7874588602113286


Получили метрику, которая выше 0,75. Данная модель решает поставленную задачу.

## Вывод

У нас был классифицированный текст.

Мы предобработали его для обучения модели: очистили текст и лемматизировали его.

Далее обучили несколько моделей. Нужную метрику F1 показала модель логистической регрессии с параметром С=11 (0.78).